# Sesión 6.2 - BERT Finetuning para clasificación de textos

En esta segunda parte de la sesión vamos a ver como se puede reentrenar un modelo preentrenado de BERT/RoBERTa para poder conseguir un clasificador de texto como en las prácticas anteriores con el datasetEspañol.csv.

**Es importante tener en cuenta que para hacer el finetuning de modelos Transformers como BERT o RoBERTa es muy recomendable tener una GPU para poder reducir de manera drástica el tiempo de procesamiento. Por lo tanto se recomienda cargar este notebook en COLAB**

Lo primero que haremos será instalar las librerías de Huggingface transformers y datasets.


In [ ]:
# Install libraries
!pip3 install transformers datasets torch
!pip3 install accelerate evaluate

# Descargamos nuestro datasetEspañol.csv
!wget --no-check-certificate http://valencia.inf.um.es/valencia-tgine/datasetEspañol.csv

In [2]:
import transformers

# Modelo BERT en español - BETO
#path_bert_model = 'dccuchile/bert-base-spanish-wwm-uncased'
# Modelo BERT multilingüe
#path_bert_model = 'bert-base-multilingual-cased'
# Modelo BERTIN basado en RoBERTa
#path_bert_model = 'bertin-project/bertin-roberta-base-spanish'
# Modelo MarIA basado en RoBERTa
path_bert_model = 'PlanTL-GOB-ES/roberta-base-bne'

# Modelo "destilados" de BERT en español
#path_bert_model = 'CenIA/distillbert-base-spanish-uncased'

# Modelo AlBERT en español
#path_bert_model = 'CenIA/albert-base-spanish'

## Apartado 1.1 Cargarmos un modelo de BERT/RoBERTa preentrenado para clasificación binaria

A continuación vamos a cargar un modelo BERT/RoBERTa para clasificación binaria utilizando las librería KERAS de TensorFlow.

El finetuning y uso de BERT/RoBERTa se puede hacer tanto con Pytorch como con KERAS.

A continuación se muestra un ejemplo con KERAS que resulta más gráfico.

In [ ]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
import json

# Cargamos un modelo de BERT preentrenado para clasificación. El número de etiquetas es 2
NUM_LABELS = 2
# La clase TFAutoModelForSequenceClassification es de Tensorflow y la AutoModelForSequenceClassification es de Pytorch
bert_class_model = AutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS)
# Cargamos el Tokenizer
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = bert_class_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= bert_class_model.config.id2label[predicted_class_id]
  print(text,'=>', prediction)


## Apartado 1.2 Realizamos un Finetuning del modelo preentrenado de BERT/RoBERTa en Pytorch

Se puede observar por los resultados anteriores que este modelo de BERT/RoBERTA es un modelo general y no está adaptado para hacer clasificación de texto en 'positivo' y 'negativo' y mucho menos para el dominio del estado de alarma.

Vamos a cargar el dataset de siempre para poder hacer un finetuning de este modelo para mejorar de manera muy significativa los resultados del mismo.

Para el entrenamiento de los modelos de DeepLearning es necesario tener 3 conjuntos de datos: Train, Eval y Test.

In [ ]:
import pandas
df = pandas.read_csv("datasetEspañol.csv",encoding="UTF-8")

p_train = 0.80 # Porcentaje de train.
p_eval = 0.20 # Porcentaje de eval.
p_test = 0.20 # Porcentaje de test

from sklearn.model_selection import train_test_split

df = df.sample(frac=1) #mezclamos el dataset


# Ponemos en lower_case en los tweets
df.tweet = df.tweet.apply (lambda x: x.lower())


# Para poder entrenar es necesario codificar las etiquetas como números. Para eso codificaremos
# los negativos con 0 y los positivos con 1
df['_label'] = df['label'].apply(lambda x : 1 if x == 'positive' else 0)

df_train, df_test = train_test_split (df, test_size = p_test)
df_train, df_eval = train_test_split (df_train, test_size = p_eval)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

A continuación preparamos los conjuntos de entrenamiento, evaluación y test en **Pytorch** para poder hacer el entrenamiento del modelo.

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

# Cargamos el modelo para clasificación en Pytorch
bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, num_labels=NUM_LABELS)

# Los datasets se preparan dde manera distinta a Tensorflow
class TGINEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

import numpy as np
from evaluate import load

metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments

batch_train_size = 16
batch_val_size = 64

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results',
  logging_dir = './logs',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size,
  report_to="none"  # Desactiva wandb
)


tokenized_train_dataset = tokenizer (df_train.tweet.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = tokenizer (df_eval.tweet.tolist (), truncation=True, padding = True)
tokenized_test_dataset = tokenizer (df_test.tweet.tolist (), truncation=True, padding = True)


# Como antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test._label.tolist())

from transformers import Trainer

trainer = Trainer (
    model = bert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()


print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Salvamos el modelo reentrenado
modelo ='modeloReentrenadoPytorch'
bert_class_model_pytorch.save_pretrained (modelo)
tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))


Volvemos a clasificar las frases de antes y vemos que dan unos resultados mucho mejores con el entrenamiento.

In [ ]:
# Cargamos el modelo ya con el FineTuning hecho
bert_class_model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=NUM_LABELS)
# Cargamos el Tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelo)

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = bert_class_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= bert_class_model.config.id2label[predicted_class_id]
  print(text,'=>', predicted_class_id, '=>', prediction, "  ", logits.softmax(1))


## Apartado 1.3 Compatibilidad entre modelos Pytorch y Tensorflow

Se pueden cargar modelos de Pytorch en Tensorflow y viceversa. A continuación cargamos el modelo entrenado en Pytorch para evaluar las frases de los textos de antes y ver su clasificación.

In [ ]:
import tensorflow as tf

# Probar a inferir nuevas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]


tf_model = TFAutoModelForSequenceClassification.from_pretrained('modeloReentrenadoPytorch')

# Imprimimos las predicciones obtenidas
for text in textos:
  predict_input = tokenizer.encode (text, truncation=True, padding=True, return_tensors="tf")
  tf_output = tf_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  print(text,'=>', tf_prediction)

## Apartado 1.4 Probamos a hacer Finetunning de un modelo "destilado" de BERT

En el siguiente bloque de código hacemos el mismo entrenamiento en Pytorch pero con el modelo 'CenIA/distillbert-base-spanish-uncased' que es más pequeño y rápido de entrenar


In [ ]:
path_distilbert_model = 'CenIA/distillbert-base-spanish-uncased'

# Cargamos el modelo para clasificación en Pytorch
distilbert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_distilbert_model, num_labels=NUM_LABELS)
# Cargamos el tokenizer de este modelo
distilbert_tokenizer = AutoTokenizer.from_pretrained(path_distilbert_model)

# Definimos algunos training arguments como el tamaño del bach_size
training_args = transformers.TrainingArguments (
  output_dir = './results_distilbert',
  logging_dir = './logs_distilbert',
  per_device_train_batch_size = batch_train_size,
  per_device_eval_batch_size = batch_val_size,
  report_to="none"  # Desactiva wandb
)


tokenized_train_dataset = distilbert_tokenizer (df_train.tweet.tolist (),  truncation=True, padding = True)
tokenized_eval_dataset = distilbert_tokenizer (df_eval.tweet.tolist (), truncation=True, padding = True)
tokenized_test_dataset = distilbert_tokenizer (df_test.tweet.tolist (), truncation=True, padding = True)


# Como antes, las etiquetas deben ser numéricas para poder entrenar.
# Preparamos los 3 datasets para hacer el finetuning
train_dataset = TGINEDataset (tokenized_train_dataset, df_train._label.tolist())
eval_dataset = TGINEDataset (tokenized_eval_dataset, df_eval._label.tolist())
test_dataset = TGINEDataset (tokenized_test_dataset, df_test._label.tolist())

from transformers import Trainer

trainer = Trainer (
    model = distilbert_class_model_pytorch,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()


print ("PREDICCIONES SOBRE EVAL")
distilbert_class_model_pytorch.eval ()
print (json.dumps (trainer.evaluate (), indent = 2))


# Salvamos el modelo reentrenado
modelo ='modeloReentrenadoPytorchDistilbert'
distilbert_class_model_pytorch.save_pretrained (modelo)
distilbert_tokenizer.save_pretrained (modelo)

print ("PREDICCIONES SOBRE TEST")
predictions = trainer.predict (test_dataset)
print(json.dumps(predictions.metrics, indent = 2))

In [ ]:
# Mover el modelo a la GPU
distilbert_class_model_pytorch = distilbert_class_model_pytorch.to("cuda")

# Probamos a clasificar estas frases
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# TEST

for text in textos:
  # Movemos los tensores del tokenizer a la GPU
  inputs = distilbert_tokenizer(text, return_tensors="pt").to("cuda")
  with torch.no_grad():
    logits = distilbert_class_model_pytorch(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= distilbert_class_model_pytorch.config.id2label[predicted_class_id]
  print(text,'=>', predicted_class_id, '=>', prediction, "  ", logits.softmax(1))